# Collaborative Filtering

### Notations
Imagine we are to predict the rating for each movie given by a user.

| Notation | Description |
| :- | :- |
| $n_u$ | Number of users |
| $n_m$ | Number of movies |
| $r(i, j)$ | It equals 1 if user "j" has rated move "i" |
| $y^{(i, j)}$ | Rating of user "j" for the movie "i" |
| $n$ | Number of features for each movie |
| $w^{(j)}$ | Weights of features for user "j" |
| $x^{(i)}$ | Features for movie "i" |
| $b^{(j)}$ | Intersection for user "j" |

Given a dataset like below, we can use collaborative filtering to use all the users' rating to predict future ratings.

| Movie | User1 | User2 | User3 | User4 | User5 | Feature1 ($x_1$) | Feature2 ($x_2$) |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| Movie1 | 5 | 5 | 0 | 0 | ? | 9.9 | 0 |
| Movie2 | 5 | ? | ? | 0 | ? | 1.0 | 0.01 |
| Movie3 | ? | 4 | 0 | ? | ? | 0.99 | 0 |
| Movie4 | 0 | 0 | 5 | 4 | ? | 0.1 | 1.0 |
| Movie5 | 0 | 0 | 5 | ? | ? | 0 | 0.9 |

In [1]:
import numpy as np
import tensorflow as tf

### Function
The function is like a linear function with a difference that we have three variables<br>
$
f_{W, b, X}(W^{(j)}, b^{(j)}, X^{(i)}) = W^{(j)}.X^{(i)} + b^{(j)}
$
<br>
If the function is mean normalized: <br>
$
f_{W, b, X}(W^{(j)}, b^{(j)}, X^{(i)}) = W^{(j)}.X^{(i)} + b^{(j)} - \mu^{(i)}
$
<br>
where $\mu$ is a vector with $n_m$ rows. (Mean normalization could be calculated in a way to have $n_u$ rows i.e. be calculated via taking the mean of each column rather than row)

### Cost function
Now instead of learning parameters $W$ and $b$ we also have to learn $X$. <br>
$
\begin{equation}
\displaystyle
    J(W, b, X) = \frac{1}{2} \sum_{(i, j):r(i, j)=1}(W^{(j)}.X^{(i)} + b^{(j)} - y^{(i, j)})^2  + \frac{\lambda}{2}\sum_{j=1}^{n_u}\sum_{k=1}^{n}(W_{k}^{(j)})^2 + \frac{\lambda}{2}\sum_{i=1}^{n_m}\sum_{k=1}^{n}(X_{k}^{(i)})^2
\end{equation}
$
<br>
We can write this in another way <br>
$
\begin{equation}
\displaystyle
    J(W, b, X) = \frac{1}{2} \sum_{i=1}^{n_m}\sum_{j=1}^{n_u}r(i, j)\times(W^{(j)}.X^{(i)} + b^{(j)} - y^{(i, j)})^2  + \frac{\lambda}{2}\sum_{j=1}^{n_u}\sum_{k=1}^{n}(W_{k}^{(j)})^2 + \frac{\lambda}{2}\sum_{i=1}^{n_m}\sum_{k=1}^{n}(X_{k}^{(i)})^2
\end{equation}
$

### Gradient descent
Repeat {<br>
    $W_{i}^{(j)} = W_{i}^{(j)} - \alpha\displaystyle\frac{\partial}{\partial W_{i}^{(j)}}J(W, b, X)$ <br>
    $b^{(j)} = b^{(j)} - \alpha\displaystyle\frac{\partial}{\partial b^{(j)}}J(W, b, X)$ <br>
    $X_{k}^{(j)} = X_{k}^{(j)} - \alpha\displaystyle\frac{\partial}{\partial X_{k}^{(j)}}J(W, b, X)$ <br>
}

In [2]:
def cf_func(w, x, b):
    return np.dot(w, x) + b

In [3]:
def cf_cost(X, W, b, Y, R, lambda_):
    nm, nu = Y.shape
    J = 0
    nm, n = X.shape
    nu, _ = W.shape
    
    for j in range(nu):
        for i in range(nm):
            J += R[i, j] * (1/2) * (cf_func(W[j], X[i], b[0, j]) - Y[i, j]) ** 2
    
    for j in range(nu):
        for k in range(n):
            J += (lambda_ / 2 ) * W[j, k] ** 2
            
    for i in range(nm):
        for k in range(n):
            J += (lambda_ / 2) * X[i, k] ** 2

    return J

Function below is vectorized implementation of the code (refrence "Coursera, unsupervised learning, recommenders, reinforcement-learning")

In [4]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

Since finding derivative of the cost function seems difficult, we use "auto diff" tools from tensorflow.

In [5]:
def mean_normalization(X):
    mu = X.sum(axis=1) / X.shape[1]
    mu = mu.reshape(-1, 1)
    X = X - mu.reshape(-1, 1)
    return X, mu

In [6]:
def gradient_descent(X, W, b, Y, R, alpha=0.1, iterations=200, lambda_=0):
    optimizer = tf.keras.optimizers.Adam(alpha)
    for i in range(iterations):
        with tf.GradientTape() as tape:
            cost = cf_cost(X, W, b, Y, R, lambda_)
            
        grads = tape.gradient(cost, [W, X, b])
        optimizer.apply_gradients(zip(grads, W, x, b))
        
        print(f'Iteration {i} cost:{cost}')